In [1]:
import os

import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, random_split

In [2]:
torch.manual_seed(100)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
spectrogram_folder_path = "../DSD100spectrogram/Train"

In [5]:
spectrogram_path = [spectrogram_folder_path + '/'+name for name in sorted(os.listdir(spectrogram_folder_path))]

In [6]:
data = np.load(spectrogram_path[0])
mixture = data['mixture']
bass = data['bass']
drum = data['drum']
vocal = data['vocal']
instrumental = data['instrumental']

In [7]:
mixture.shape

(513, 128)

Custom Dataset


In [8]:
class DSD100(Dataset):
    def __init__(self, data_path):
        self.spectrogram_path = [data_path + '/'+name for name in sorted(os.listdir(data_path))]
    
    def __len__(self):
        return len(self.spectrogram_path)
    
    def __getitem__(self, index):
        data = np.load(self.spectrogram_path[index])
        mixture = data['mixture'][np.newaxis,:511,:127]
        bass = data['bass'][np.newaxis,:511,:127]
        drum = data['drum'][np.newaxis,:511,:127]
        vocal = data['vocal'][np.newaxis,:511,:127]
        instrumental = data['instrumental'][np.newaxis,:511,:127]
        return (
            torch.from_numpy(np.copy(mixture)).to(device),
            torch.from_numpy(np.copy(bass)).to(device),
            torch.from_numpy(np.copy(drum)).to(device),
            torch.from_numpy(np.copy(vocal)).to(device),
            torch.from_numpy(np.copy(instrumental)).to(device)
        )


In [9]:
dataset = DSD100('../DSD100spectrogram') 

In [10]:
for mixture, bass, drum, vocal, instrumental in dataset:
    print(mixture.shape)
    print(bass.shape)
    print(drum.shape)
    print(vocal.shape)
    print(instrumental.shape)
    break

torch.Size([1, 513, 128])
torch.Size([1, 513, 128])
torch.Size([1, 513, 128])
torch.Size([1, 513, 128])
torch.Size([1, 513, 128])


In [11]:
len(dataset)

7548

random split dataset

In [12]:
def data_split(dataset, train_ratio):
    return random_split(dataset, [train_ratio, 1 - train_ratio])


In [ ]:
train_dataset, val_dataset = data_split(dataset, 0.7)

In [14]:
len(train_dataset)

6039

In [13]:
len(val_dataset)

1509

dataloader

In [ ]:
def dataloader(dataset, batch_size = 64, shuffle = False):
    return DataLoader(dataset, batch_size = batch_size, shuffle = shuffle)

In [ ]:
train_dataloader = dataloader(train_dataset, 128, shuffle = True)
val_dataloader = dataloader(val_dataset, 128, shuffle = False)

for mixture, bass, drum, vocal, instrumental in train_dataloader:
    print(mixture.shape)
    print(bass.shape)
    print(drum.shape)
    print(vocal.shape)
    print(instrumental.shape)
    break

use Data.py for making dataset, random_split and DataLoader

In [1]:
from CustomDataset.Data import (
    DSD100,
    data_split,
    dataloader
)

In [2]:
dataset = DSD100('../DSD100spectrogram')

In [3]:
for mixture, bass, drum, vocal, instrumental in dataset:
    print(mixture.shape)
    print(bass.shape)
    print(drum.shape)
    print(vocal.shape)
    print(instrumental.shape)
    break

torch.Size([1, 511, 127])
torch.Size([1, 511, 127])
torch.Size([1, 511, 127])
torch.Size([1, 511, 127])
torch.Size([1, 511, 127])


In [4]:
len(dataset)

10284

In [5]:
train_dataset, val_dataset = data_split(dataset, 0.7)

In [6]:
len(train_dataset)

7199

In [7]:
len(val_dataset)

3085

In [8]:
train_dataloader = dataloader(train_dataset, 128, shuffle = True)
val_dataloader = dataloader(val_dataset, 128, shuffle = False)

In [9]:
for mixture, bass, drum, vocal, instrumental in train_dataloader:
    print(mixture.shape)
    print(bass.shape)
    print(drum.shape)
    print(vocal.shape)
    print(instrumental.shape)
    break

torch.Size([128, 1, 511, 127])
torch.Size([128, 1, 511, 127])
torch.Size([128, 1, 511, 127])
torch.Size([128, 1, 511, 127])
torch.Size([128, 1, 511, 127])
